### Data Overview and Preparation
The dataset combines multiple energy indicators (Electricity Installed Capacity, etc.) 
across >150 countries and 20+ years, covering Solar, Wind, Hydro, Bio, and Fossil energy. 
Data cleaning steps:
1. Standardized technology names (e.g., 'Hydropower (excl. Pumped Storage)' → 'Hydro')
2. Converted all year columns (F2000–F2022) to numeric
3. Filtered to comparable units (Megawatt, MW)
These steps ensure comparability across regions and over time.


In [30]:
import pandas as pd

# --- 1️⃣ Load EV dataset ---
df_ev = pd.read_csv("Isea/Global_EV_clean_ready.csv")
year_cols = [c for c in df_ev.columns if isinstance(c,str) and c.startswith("F")]

# --- 2️⃣ Load or define population data (static millions → individuals) ---
pop_data = [
    ("CHN", 1425), ("IND", 1410), ("USA", 339), ("IDN", 277), ("PAK", 241),
    ("BRA", 216), ("NGA", 223), ("RUS", 144), ("JPN", 125), ("MEX", 128),
    ("DEU", 84), ("FRA", 65), ("GBR", 67), ("ITA", 59), ("CAN", 40),
    ("KOR", 52), ("AUS", 26), ("ESP", 48), ("POL", 38), ("ZAF", 61),
    ("SAU", 36), ("TUR", 86), ("IRN", 86), ("EGY", 112), ("ARG", 46),
    ("COL", 52), ("THA", 71), ("VNM", 100), ("SWE", 10), ("NOR", 5),
    ("CHE", 9), ("NLD", 18), ("BEL", 12), ("AUT", 9), ("FIN", 6),
    ("DNK", 6), ("SGP", 6), ("ISR", 9), ("NZL", 5), ("PRT", 10),
]
pop_df = pd.DataFrame(pop_data, columns=["ISO3", "Pop2023"])
pop_df["Pop2023"] *= 1_000_000

# --- 3️⃣ Compute EV_total robustly ---
ev_types = ["BEV","PHEV","FCEV"]
present = set(df_ev["Technology"].unique())
if any(t in present for t in ev_types):
    # sum the EV types we care about
    df_ev_sum = (
        df_ev[df_ev["Technology"].isin(ev_types)]
        .groupby(["Country", "ISO3"], as_index=False)[year_cols]
        .sum()
    )
    df_ev_sum["Technology"] = "EV_total"
elif "Total" in present:
    # fall back to Total rows if they represent vehicle totals
    df_ev_sum = df_ev[df_ev["Technology"] == "Total"][ ["Country","ISO3"] + year_cols ].copy()
    df_ev_sum["Technology"] = "EV_total"
else:
    # last-resort: sum across all technology rows per country (may include non-vehicle rows)
    df_ev_sum = (
        df_ev.groupby(["Country", "ISO3"], as_index=False)[year_cols]
        .sum()
    )
    df_ev_sum["Technology"] = "EV_total"

# --- 4️⃣ Merge population and compute per-capita ---
df_merge = df_ev_sum.merge(pop_df, on="ISO3", how="left")

# diagnostics
total_countries = len(df_merge)
missing_pop = df_merge[ df_merge["Pop2023"].isna() ]
print(f"Computed EV_total rows: {total_countries}")
print(f"Countries missing population: {len(missing_pop)}")
if len(missing_pop) > 0:
    print(missing_pop[["Country","ISO3"]].to_string(index=False))

# compute per-capita (EV count per person)
for c in year_cols:
    # avoid division by zero / NaN: result will be NaN where Pop2023 is missing
    df_merge[c] = df_merge[c] / df_merge["Pop2023"]

# --- 5️⃣ Final cleanup and save ---
df_merge = df_merge[["Country", "ISO3"] + year_cols]
df_merge["Technology"] = "EVs_per_capita"
df_out = df_merge[["Country", "ISO3", "Technology"] + year_cols]
df_out.to_csv("Isea/Global_EV_percapita.csv", index=False)

print("✅ Saved: Isea/Global_EV_percapita.csv")
print(df_out.head(5))

Computed EV_total rows: 49
Countries missing population: 20
             Country ISO3
            Bulgaria  BUL
               Chile  CHL
          Costa Rica  COS
             Croatia  CRO
              Cyprus  CYP
      Czech Republic  CZE
             Estonia  EST
              Greece  GRC
             Hungary  HUN
             Iceland  ICE
             Ireland  IRE
              Latvia  LAT
           Lithuania  LIT
          Luxembourg  LUX
   Rest of the world  RES
             Romania  ROU
          Seychelles  SEY
            Slovakia  SVK
            Slovenia  SLO
United Arab Emirates  ARE
✅ Saved: Isea/Global_EV_percapita.csv
     Country ISO3      Technology         F2010         F2011         F2012  \
0  Australia  AUS  EVs_per_capita  0.000000e+00  3.769231e-06  2.115385e-05   
1    Austria  AUT  EVs_per_capita  3.888889e-05  1.100000e-04  1.711111e-04   
2    Belgium  BEL  EVs_per_capita  1.675000e-05  6.308333e-05  2.242500e-04   
3     Brazil  BRA  EVs_per_capita  4.629

In [31]:
import pandas as pd
import ipywidgets as widgets
import inspect
from IPython.display import display, clear_output
from Isea.worldmaplinechart import WorldMapLineChart

# --- Load (or defer) datasets ---
# Note: heavy / repeat reads are fine in notebook; other cells compute per-capita CSV.
df_energy = pd.read_csv("Isea/Energy_clean.csv")
df_ev = pd.read_csv("Isea/Global_EV_clean_ready.csv")

# helper to build optional kwargs for WorldMapLineChart without raising if unsupported
def _chart_kwargs_for_hover():
    kwargs = {}
    try:
        sig = inspect.signature(WorldMapLineChart.__init__)
        params = sig.parameters
        if 'hover_template' in params:
            kwargs['hover_template'] = "%{value:.2e}"
        elif 'hovertext' in params:
            # some APIs may accept hovertext array; we'll not set it here by default
            kwargs['hovertext'] = None
    except Exception as e:
        # if introspection fails, return empty kwargs
        pass
    return kwargs

# --- Widget factory: energy ---
def make_energy_widget():
    year_cols = [c for c in df_energy.columns if isinstance(c, str) and c.startswith("F")]
    tech_map = {
        "Hydropower (excl. Pumped Storage)": "Hydro",
        "Solar energy": "Solar",
        "Wind energy": "Wind",
        "Bioenergy": "Bio",
        "Fossil fuels": "Fossil",
    }
    kwargs = _chart_kwargs_for_hover()
    if kwargs.get('hovertext') is None:
        kwargs.pop('hovertext', None)

    return WorldMapLineChart(
        df_energy,
        year_cols=year_cols,
        country_col="Country",
        iso3_col="ISO3",
        tech_col="Technology",
        tech_keep=["Solar","Wind","Hydro","Bio","Fossil"],
        tech_map=tech_map,
        share_numerator=["Solar","Wind","Hydro","Bio"],
        title="Renewables share of installed capacity by country",
        subtitle="Move the year slider. Click countries to compare.",
        share_label="Renewables share",
        **kwargs,
    )

# --- Widget factory: EV counts ---
def make_ev_widget():
    df_ev2 = pd.read_csv("Isea/Global_EV_clean_ready.csv")
    year_cols = [c for c in df_ev2.columns if isinstance(c, str) and c.startswith("F")]

    present = set(df_ev2["Technology"].astype(str).str.strip().unique())
    tech_keep = [t for t in ["BEV","PHEV","FCEV","Other","ICE"] if t in present]
    if "Total" in present and "Total" not in tech_keep:
        tech_keep.append("Total")

    # Force BEV to be plotted as an absolute series when present
    if "BEV" in present:
        share_numerator = ["BEV"]
        share_denominator = ["BEV"]
    else:
        share_numerator = [t for t in ["BEV","PHEV","FCEV"] if t in present]
        share_denominator = ["Total"] if "Total" in present else tech_keep.copy()

    kwargs = _chart_kwargs_for_hover()
    if kwargs.get('hovertext') is None:
        kwargs.pop('hovertext', None)

    return WorldMapLineChart(
        df_ev2,
        year_cols=year_cols,
        country_col="Country",
        iso3_col="ISO3",
        tech_col="Technology",
        tech_keep=tech_keep,
        tech_map={},                       # already normalized
        share_numerator=share_numerator,   # BEV (absolute series if available)
        share_denominator=share_denominator,
        title="BEV count by country",
        subtitle="Line shows BEV counts (absolute) over years.",
        share_label="BEV",                # ensures absolute scaling in frontend
        **kwargs,
    )

# --- Widget factory: EV per capita ---
def make_ev_percapita_widget():
    df_pc = pd.read_csv("Isea/Global_EV_percapita.csv")
    year_cols = [c for c in df_pc.columns if isinstance(c, str) and c.startswith("F")]
    kwargs = _chart_kwargs_for_hover()
    if kwargs.get('hovertext') is None:
        kwargs.pop('hovertext', None)

    return WorldMapLineChart(
        df_pc,
        year_cols=year_cols,
        country_col="Country",
        iso3_col="ISO3",
        tech_col="Technology",
        tech_keep=["EVs_per_capita"],
        share_numerator=["EVs_per_capita"],
        share_denominator=["EVs_per_capita"],
        title="EVs per Person by Country",
        subtitle="Shows the number of electric vehicles per person (EVs ÷ population).",
        share_label="EVs per person",
        **kwargs,
    )

# --- Unified selector widget (single place in notebook) ---
ev_menu = ["BEV","EV","FCEV","PHEV","fast","slow"]
dataset_selector = widgets.Dropdown(
    options=["Renewable Energy", "EVs (counts)", "EVs per Capita"] + ev_menu,
    value="Renewable Energy",
    description="Dataset:",
    style={"description_width": "initial"},
    layout=widgets.Layout(width="420px"),
    )

output = widgets.Output()

def _display_filtered(selection):
    # use the robust helper if available; otherwise compute with normalization locally
    try:
        df_f = _get_filtered_ev_df([selection])
        # try to reuse precomputed year columns when present
        year_cols = _year_cols_ev
    except Exception:
        df_all = pd.read_csv("Isea/Global_EV_clean_ready.csv")
        year_cols = [c for c in df_all.columns if isinstance(c,str) and c.startswith("F")]
        tech_norm = df_all['Technology'].astype(str).str.strip().str.upper()
        key = str(selection).strip().upper()
        if key == 'EV':
            df_f = df_all[tech_norm.isin(['BEV','PHEV','FCEV'])]
        elif key in ('BEV','PHEV','FCEV'):
            df_f = df_all[tech_norm == key]
        elif key in ('FAST','SLOW'):
            cols = [c for c in df_all.columns if any(k in c.lower() for k in ('charge','charger','speed','type','fast','slow'))]
            if cols:
                masks = [df_all[c].astype(str).str.contains(key, case=False, na=False) for c in cols]
                mask = masks[0] if masks else pd.Series([False]*len(df_all))
                for m in masks[1:]:
                    mask = mask | m
                df_f = df_all[mask]
            else:
                df_f = df_all.iloc[0:0]
        else:
            df_f = df_all.iloc[0:0]

    with output:
        clear_output(wait=True)
        print(f"Selection: {selection} → filtered rows: {len(df_f)}")
        if df_f.empty:
            print("No rows match selection — showing full EV chart instead.")
            display(make_ev_widget())
            return

        present = set(df_f['Technology'].astype(str).str.strip().unique())
        tech_keep = [t for t in ["BEV","PHEV","FCEV","Other","ICE"] if t in present]
        if "Total" in present and "Total" not in tech_keep:
            tech_keep.append("Total")
        if "BEV" in present:
            share_numerator = ["BEV"]
            share_denominator = ["BEV"]
            share_label = "BEV"
        else:
            share_numerator = [t for t in ["BEV","PHEV","FCEV"] if t in present]
            share_denominator = ["Total"] if "Total" in present else tech_keep.copy()
            share_label = "EVs"

        try:
            chart = WorldMapLineChart(
                df_f,
                year_cols=year_cols,
                country_col="Country",
                iso3_col="ISO3",
                tech_col="Technology",
                tech_keep=tech_keep,
                tech_map={},
                share_numerator=share_numerator,
                share_denominator=share_denominator,
                title=f"Filtered EVs: {selection}",
                subtitle="Filtered selection",
                share_label=share_label,
            )
            display(chart)
        except Exception as e:
            print("Error creating filtered chart, falling back to full chart:", e)
            display(make_ev_widget())

def on_select(change):
    # handle both widget events and manual dict trigger
    new = change.get("new") if isinstance(change, dict) else change.new
    if new is None:
        return
    if new == "Renewable Energy":
        with output:
            clear_output(wait=True)
            display(make_energy_widget())
    elif new == "EVs (counts)":
        with output:
            clear_output(wait=True)
            display(make_ev_widget())
    elif new == "EVs per Capita":
        with output:
            clear_output(wait=True)
            display(make_ev_percapita_widget())
    elif new in ev_menu:
        _display_filtered(new)
    else:
        with output:
            clear_output(wait=True)
            print(f"Unknown selection: {new}")

dataset_selector.observe(on_select, names="value")
display(dataset_selector, output)

# Trigger initial load
on_select({"new": dataset_selector.value})

Dropdown(description='Dataset:', layout=Layout(width='420px'), options=('Renewable Energy', 'EVs (counts)', 'E…

Output()

In [ ]:
# --- EV filter UI: BEV, EV, FCEV, PHEV, fast, slow (robust matching) ---
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

# read EV dataset and year columns (used to build charts)
_df_ev_master = pd.read_csv("Isea/Global_EV_clean_ready.csv")
_year_cols_ev = [c for c in _df_ev_master.columns if isinstance(c,str) and c.startswith("F")]

ev_options = ["BEV","EV","FCEV","PHEV","fast","slow"]
ev_checks = {opt: widgets.Checkbox(value=False, description=opt) for opt in ev_options}
ev_box = widgets.VBox([widgets.HBox([ev_checks[o] for o in ev_options[:3]]), widgets.HBox([ev_checks[o] for o in ev_options[3:]])])
ev_out = widgets.Output()

def _normalize_tech_series(df):
    # return normalized uppercase stripped series for comparison (non-destructive)
    return df['Technology'].astype(str).str.strip().str.upper()

def _get_filtered_ev_df(selected):
    df = pd.read_csv("Isea/Global_EV_clean_ready.csv")
    tech_norm = _normalize_tech_series(df)
    if not selected:
        return df
    parts = []
    for s in selected:
        key = str(s).strip().upper()
        if key == 'EV':
            parts.append(df[tech_norm.isin(['BEV','PHEV','FCEV'])])
        elif key in ('BEV','PHEV','FCEV'):
            parts.append(df[tech_norm == key])
        elif key in ('FAST','SLOW'):
            # attempt to find charger/speed-like columns and filter by substring case-insensitive
            cols = [c for c in df.columns if any(k in c.lower() for k in ('charge','charger','speed','type','fast','slow'))]
            if cols:
                masks = [df[c].astype(str).str.contains(key, case=False, na=False) for c in cols]
                mask = masks[0] if masks else pd.Series([False]*len(df))
                for m in masks[1:]:
                    mask = mask | m
                parts.append(df[mask])
            else:
                parts.append(df.iloc[0:0])
        else:
            parts.append(df.iloc[0:0])
    if parts:
        out_df = pd.concat(parts, ignore_index=True).drop_duplicates().reset_index(drop=True)
    else:
        out_df = df.iloc[0:0]
    return out_df

def _update_ev_view(change=None):
    selected = [k for k,v in ev_checks.items() if v.value]
    with ev_out:
        clear_output(wait=True)
        try:
            current = dataset_selector.value
        except NameError:
            current = None
        if current != "EVs (counts)":
            print("EV filters active only when 'EVs (counts)' is selected in the main selector.")
            display(ev_box)
            return
        print("Selected filters:", selected)
        df_f = _get_filtered_ev_df(selected)
        print(f"Filtered rows: {len(df_f)}")
        if df_f.empty:
            print("No rows match selection — showing full EV chart instead.")
            display(make_ev_widget())
            return
        # determine plotting params similar to make_ev_widget()
        present = set(df_f['Technology'].astype(str).str.strip().unique())
        tech_keep = [t for t in ["BEV","PHEV","FCEV","Other","ICE"] if t in present]
        if "Total" in present and "Total" not in tech_keep:
            tech_keep.append("Total")
        if "BEV" in present:
            share_numerator = ["BEV"]
            share_denominator = ["BEV"]
            share_label = "BEV"
        else:
            share_numerator = [t for t in ["BEV","PHEV","FCEV"] if t in present]
            share_denominator = ["Total"] if "Total" in present else tech_keep.copy()
            share_label = "EVs"
        try:
            chart = WorldMapLineChart(
                df_f,
                year_cols=_year_cols_ev,
                country_col="Country",
                iso3_col="ISO3",
                tech_col="Technology",
                tech_keep=tech_keep,
                tech_map={},
                share_numerator=share_numerator,
                share_denominator=share_denominator,
                title="Filtered EVs",
                subtitle="Filtered by checkbox selection",
                share_label=share_label,
            )
            display(chart)
        except Exception as e:
            print("Error creating filtered chart, falling back to full chart:", e)
            display(make_ev_widget())

# wire observers
for cb in ev_checks.values():
    cb.observe(_update_ev_view, names='value')
# Also update when main selector changes to EVs (counts) so the filter shows results immediately
try:
    dataset_selector.observe(lambda ch: _update_ev_view(), names='value')
except NameError:
    pass

display(widgets.Label("EV Filters (toggle to filter the EV map):"))
display(ev_box)
display(ev_out)

# initial state
_update_ev_view()

Label(value='EV Filters (toggle to filter the EV map):')

Output()

In [33]:
import pandas as pd
df_pc = pd.read_csv("Isea/Global_EV_percapita.csv")
# rows with any missing value (e.g., missing population or missing F-year columns)
missing_rows = df_pc[df_pc.isna().any(axis=1)]
print(f"Rows total: {len(df_pc)}, rows with missing values: {len(missing_rows)}")
print(missing_rows[["Country","ISO3"]].to_string(index=False))

Rows total: 49, rows with missing values: 20
             Country ISO3
            Bulgaria  BUL
               Chile  CHL
          Costa Rica  COS
             Croatia  CRO
              Cyprus  CYP
      Czech Republic  CZE
             Estonia  EST
              Greece  GRC
             Hungary  HUN
             Iceland  ICE
             Ireland  IRE
              Latvia  LAT
           Lithuania  LIT
          Luxembourg  LUX
   Rest of the world  RES
             Romania  ROU
          Seychelles  SEY
            Slovakia  SVK
            Slovenia  SLO
United Arab Emirates  ARE


In [34]:
import pandas as pd
df = pd.read_csv("Isea/Global_EV_clean_ready.csv")
vals = sorted(set(str(x).strip() for x in df["Technology"].dropna().unique()))
print("Technology unique values (sample 50):", vals[:50])
print("Total rows:", len(df))
df.head()

Technology unique values (sample 50): ['BEV', 'EV', 'FCEV', 'PHEV', 'fast', 'slow']
Total rows: 240


,Country,ISO3,Technology,F2010,F2011,F2012,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021,F2022,F2023
0,Australia,AUS,BEV,NaN,98.00000,390.0000,600.0000,1150.000,2260.000,2870.000,4600.000,7000.000,18300.00,22200.00,51000.00,100000.00,237000.0
1,Australia,AUS,EV,NaN,0.00689,0.0324,0.0386,0.174,0.227,0.187,0.312,0.497,1.34,1.28,3.12,5.69,13.2
2,Australia,AUS,FCEV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.00,115.00,71.0
3,Australia,AUS,PHEV,NaN,NaN,160.0000,280.0000,2050.000,3100.000,3500.000,5000.000,7500.000,11500.00,11700.00,17400.00,26900.00,42000.0
4,Australia,AUS,fast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.000,61.000,NaN,NaN,320.00,470.00,660.0


In [35]:
cols = [c for c in df.columns if any(k in c.lower() for k in ("charge","charger","speed","fast","slow","type"))]
print("Columns that may contain charger/speed info:", cols)

Columns that may contain charger/speed info: []


In [ ]:
df_bev = df[df["Technology"].astype(str).str.lower().str.contains("bev", na=False)]
print("BEV rows:", len(df_bev))
display(df_bev.head())

BEV rows: 49


,Country,ISO3,Technology,F2010,F2011,F2012,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021,F2022,F2023
0,Australia,AUS,BEV,NaN,98.0,390.0,600.0,1150.0,2260.0,2870.0,4600.0,7000.0,18300.0,22200.0,51000.0,100000.0,237000.0
6,Austria,AUT,BEV,350.0,990.0,1400.0,2750.0,5100.0,6700.0,12900.0,20400.0,27800.0,39300.0,61000.0,110000.0,144000.0,208000.0
12,Belgium,BEL,BEV,183.0,714.0,2000.0,2250.0,4141.0,5357.0,8151.0,11134.0,15913.0,29713.0,49382.0,78386.0,134160.0,294980.0
18,Brazil,BRA,BEV,10.0,8.0,22.0,51.0,135.0,189.0,391.0,489.0,667.0,1676.0,2710.0,8880.0,26490.0,58300.0
24,Bulgaria,BUL,BEV,NaN,NaN,NaN,NaN,NaN,21.0,5.0,68.0,190.0,180.0,140.0,430.0,1000.0,1800.0


: 

### Analytical Interpretation
By comparing renewable capacity growth with EV adoption:
- Countries with early renewable expansion (e.g., Germany, China) tend to show earlier EV adoption surges.
- Oil-dependent countries lag in both indicators, highlighting infrastructural dependence.
- The visualization suggests a feedback loop: as renewable capacity grows, EVs become more viable (charging infrastructure and perception).
